In [92]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from dotenv import dotenv_values
from sqlalchemy import create_engine
from sklearn import datasets  
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import sklearn.metrics
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
import statsmodels.formula.api as sm
import scipy.stats as stats
from PIL import Image
import plotly.express as px
import plotly.offline as pyo
import plotly.figure_factory as pf
import os
import streamlit as sl

### Reading different datasets 

### Filtering Movie titles

### Reading sorted Data

In [2]:
n= pd.read_csv("../data/movie_sorted.csv")

In [3]:
n.tail()

,movieid,year,title
17765,17766,2002,Where the Wild Things Are and Other Maurice Se...
17766,17767,2004,Fidel Castro: American Experience
17767,17768,2000,Epoch
17768,17769,2003,The Company
17769,17770,2003,Alien Hunter


### Reading the rating files for movieid 1 and cleaning the file 

In [4]:
df = pd.read_csv("../data/mv_0000001.txt", names=["customerid","rating","date"] )

In [5]:
df.head()

,customerid,rating,date
0,1:,NaN,NaN
1,1488844,3.0,2005-09-06
2,822109,5.0,2005-05-13
3,885013,4.0,2005-10-19
4,30878,4.0,2005-12-26


### checking the first cleaned rating file 

In [6]:
df= pd.read_csv("../data/movieid1_rating.csv")

In [7]:
df.head()

,customerid,rating,date,movieid
0,1488844,3.0,2005-09-06,1
1,822109,5.0,2005-05-13,1
2,885013,4.0,2005-10-19,1
3,30878,4.0,2005-12-26,1
4,823519,3.0,2004-05-03,1


### Creating a function to read and clean all the rating files

### Creating and checking a test version of concatinated rating files

In [9]:
df_s= pd.read_csv("../data/ratings_small.csv", names=["customerid","rating","date", "movieid"])

In [10]:
df_s["date"] = pd.to_datetime(df_s["date"], format = "%Y-%m-%d")

In [11]:
df_s.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5005 entries, 0 to 5004
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   customerid  5005 non-null   int64         
 1   rating      5005 non-null   float64       
 2   date        5005 non-null   datetime64[ns]
 3   movieid     5005 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 156.5 KB


### Creating the large file containing all the ratings

### Reading and transforming the Qualify  file to add the Machine Learing results

### Reading the cleaned qualify file 

In [34]:
qualifier= pd.read_csv("../data/qualify.csv")

In [35]:
qualifier

,customerid,date,movieid
0,1046323,2005-12-19,1
1,1080030,2005-12-23,1
2,1830096,2005-03-14,1
3,368059,2005-05-26,1
4,802003,2005-11-07,1
...,...,...,...
2817126,1521720,2005-12-07,9998
2817127,1363704,2005-10-01,9998
2817128,2153561,2005-11-15,9999
2817129,1490137,2005-12-22,9999


### Cleaning the probe file to compare the results of qualify dataset created with help of machine learning Algorithms

### Reading the cleaned probe file

In [88]:
probability = pd.read_csv("../data/probe.csv")

In [89]:
probability

,customerid,movieid
0,30878,1
1,2647871,1
2,1283744,1
3,2488120,1
4,317050,1
...,...,...
1408390,2328701,9997
1408391,1288730,9998
1408392,2536567,9998
1408393,1107317,9998


## NEXT TASKS

#### maybe create some visualisations to check the data distribution 

#### Filter the data to create a better machine learning program

#### Decide a Machine learning technique

#### Test the algorithm

#### Add ratings in Qualify dataset and compare with probe to check the accuracy

#### Create a Predictor system which takes 3 inputs from user and provides list of related movies 

### Create a GUI using Streamlit